# SOLID Principles

## Open/Closed Principle

### Order class before changes

![order class before changes](docs/diagrams/out/__WorkspaceFolder__/various_modules/docs/diagrams/src/order_shipping/OrderShippingBefore.png)

In [1]:
from dataclasses import dataclass
from datetime import date


@dataclass
class Item:
    name: str
    price: int
    weight: int


class Order:
    def __init__(self, items: list[Item], shipping: str):
        self.line_items = items
        self._shipping_type = shipping
        self.shipping_date = date.today()

    @property
    def total(self) -> int:
        return sum(item.price for item in self.line_items)

    @property
    def total_weight(self) -> int:
        return sum(item.weight for item in self.line_items)

    @property
    def shipping(self) -> str:
        return self._shipping_type

    @shipping.setter
    def shipping(self, ship_type: str) -> None:
        self._shipping_type = ship_type

    def get_shipping_cost(self) -> int:
        if self.shipping == "ground":
            # Free ground delivery on big orders
            if self.total > 100:
                return 0
            # $1.5 per kilogram, but $10 minimum
            return max(10, self.total_weight * 1.5)

        if self.shipping == "air":
            # $3 per kilogram, but $20 minimum
            return max(20, self.total_weight * 3)

In [4]:
items = [
    Item("Sturdy Chair", 230, 40),
    Item("Amazing Desk", 400, 100),
    Item("Bright Lamp", 50, 4),
]

order = Order(items, "ground")
assert order.get_shipping_cost() == 0
assert order.total == 230 + 400 + 50
assert order.total_weight == 40 + 100 + 4
print("All unit tests have passed.")

All unit tests have passed.


### Order Class After Changes

![order and shipping classes after changes](docs/diagrams/out/__WorkspaceFolder__/various_modules/docs/diagrams/src/order_shipping/OrderShippingAfter.png)

In [3]:
from datetime import date
from dataclasses import dataclass
from abc import ABC, abstractmethod

@dataclass
class Item:
    name: str
    price: int
    weight: int


class Order:
    def __init__(self, items: list[Item], shipping: "Shipping"):
        self.line_items = items
        self.shipping = shipping
        self.shipping_date = date.today()

    @property
    def total(self) -> int:
        return sum(item.price for item in self.line_items)

    @property
    def total_weight(self) -> int:
        return sum(item.weight for item in self.line_items)

    def get_shipping_cost(self) -> float:
        return self.shipping.get_cost(self)

class Shipping(ABC):
    @abstractmethod
    def get_cost(order: Order) -> float:
        ...

    @abstractmethod
    def get_date(order: Order) -> date:
        ...

class Ground(Shipping):
    def get_cost(self, order: Order) -> float:
        if order.total > 100:
            return 0
        return max(10, order.total_weight * 1.5)

    def get_date(self, order: Order) -> date:
        return order.shipping_date


class Air(Shipping):
    def get_cost(self, order: Order) -> float:
        return max(20, order.total_weight * 3)

    def get_date(self, order: Order) -> date:
        return order.shipping_date

In [2]:
items = [
    Item("Sturdy Chair", 230, 40),
    Item("Amazing Desk", 400, 100),
    Item("Bright Lamp", 50, 4),
]

order = Order(items, Ground())
assert (ship_cost := order.get_shipping_cost()) >= 0
print("Shipping cost: ", ship_cost)
assert order.total == 230 + 400 + 50
assert order.total_weight == 40 + 100 + 4
print("All unit tests have passed.")

Shipping cost:  0
All unit tests have passed.


## Liskov Substitution Principle

### LSP: Parameter Types
> Parameter types in a method of a subclass should match or be more abstract than parameter types in the method of the superclass.

In [24]:
# Base code

class Animal:
    def __init__(self, legs=4, stamina=100):
        self.nb_legs: int = legs
        self.stamina: int = stamina

    def dance(self):
        print(f"Dances using {self.nb_legs} legs!")

class Cat(Animal):
    def __init__(self, sound="purr"):
        super().__init__(legs=4, stamina=50)
        self.sound: str = sound

@dataclass
class Food:
    energy: int

    def feed(self, cat: Cat) -> None:
        cat.stamina += self.energy
        print(cat.sound)


In [25]:
# Client code example
def feed_and_print(cat: Cat, food: Food) -> None:
    print("Before feeding, stamina: ", cat.stamina)
    food.feed(cat)
    print("After feeding, stamina: ", cat.stamina)

cat = Cat()
sausage = Food(10)
feed_and_print(cat, sausage)

Before feeding, stamina:  50
purr
After feeding, stamina:  60


**Good extension:** the new method that the subclass overrides is able to deal with more general input than the original one.

I will **not** break any client code.

In [26]:
class DancingFood(Food):
    def feed(self, animal: Animal) -> None:
        animal.stamina += self.energy
        animal.dance()

In [27]:
# Client code
cat = Cat()
fish = DancingFood(10)
feed_and_print(cat, fish)

Before feeding, stamina:  50
Dances using 4 legs!
After feeding, stamina:  60


**Bad extension:** the new method **will break** existing client code, since it no longer serves generic cats.

In [28]:
class BengalCat(Cat):
    def __init__(self, fanciness: int = 9001):
        self.fanciness_score = fanciness

class FancyFood(Food):
    def feed(self, fancy_cat: BengalCat) -> None:
        super().feed(fancy_cat)
        print(f"This cat has a fanciness of", fancy_cat.fanciness_score)

In [29]:
# Client code
cat = Cat()
fancy_feast = FancyFood(10)
feed_and_print(cat, fancy_feast)

Before feeding, stamina:  50
purr


AttributeError: 'Cat' object has no attribute 'fanciness_score'

### LSP: Return Types
> The return type in a method of a subclass should match or be a subtype of the return type in the method of the superclass.

In [40]:
# Base code

class Animal:
    def __init__(self, price: int, sound: str):
        self.price = price
        self.sound = sound

class Cat(Animal):
    def __init__(self):
        super().__init__(price=100, sound="purr")

    def be_cute(self):
        print("Passive cuteness always ON. Nothing to do in particular")

@dataclass
class CatBuyer:
    money: int = 300

    def buy_cat(self) -> Cat:
        cat = Cat()
        self.money -= cat.price
        return cat

In [42]:
# Client code

def buy_and_print(buyer: CatBuyer) -> Cat:
    print("Money before buying:", buyer.money)
    cat = buyer.buy_cat()
    print("Money after buying:", buyer.money)
    return cat

buyer = CatBuyer(money=300)
cat = buy_and_print(buyer)
print(cat.sound if cat.sound == "purr" else "PuRrR ?!!!")
cat.be_cute()

Money before buying: 300
Money after buying: 200
purr
Passive cuteness always ON. Nothing to do in particular


**Good extension:** the new method that the subclass overrides *returns a more specific output* than its subclass.

I will **not** break any client code.

In [43]:
class BengalCat(Cat):
    def __init__(self, fanciness: int = 9001):
        super().__init__()
        self.fanciness_score = fanciness
        self.price = 8000

class FancyBuyer(CatBuyer):
    def __init__(self):
        super().__init__(money=9001)

    def buy_cat(self) -> BengalCat:
        cat = BengalCat()
        self.money -= cat.price
        print("I'm ruined, but at least I have a fancy cat ┬──┬◡ﾉ(° -°ﾉ)")
        return cat

In [44]:
# Client code
buyer = FancyBuyer()
cat = buy_and_print(buyer)
print(cat.sound if cat.sound == "purr" else "PuRrR ?!!!")
cat.be_cute()

Money before buying: 9001
I'm ruined, but at least I have a fancy cat ┬──┬◡ﾉ(° -°ﾉ)
Money after buying: 1001
purr
Passive cuteness always ON. Nothing to do in particular


**Bad extension:** the new method **will break** existing client code, since it can now deliver any Animal, while the client code expects Cats.

In [46]:
class BlindBuyer(CatBuyer):
    def __init__(self):
        super().__init__(money=500)

    def buy_cat(self) -> Animal:
        cat = Animal(300, "???")
        self.money -= cat.price
        print("I don't know what I'm doing. But I have a cat (?!!)")
        return cat


In [48]:
buyer = BlindBuyer()
cat = buy_and_print(buyer)
print(cat.sound if cat.sound == "purr" else "PuRrR ?!!!")
cat.be_cute()

Money before buying: 500
I don't know what I'm doing. But I have a cat (?!!)
Money after buying: 200
PuRrR ?!!!


AttributeError: 'Animal' object has no attribute 'be_cute'

### LSP: Exception Types
> A method in a subclass shouldn’t throw types of exceptions which the base method isn’t expected to throw.


In [73]:
# Base code
class BankruptError(Exception):
    ...

@dataclass
class CatBuyer:
    money: int = 300

    def buy_cat(self) -> Cat:
        cat = Cat()
        if self.money < cat.price:
            raise BankruptError("Sorry, you're broke...")
        self.money -= cat.price
        return cat

    def beg_for_money(self, amount: int) -> None:
        print("Your bank despise you.")
        self.money += amount

In [79]:
# Client code
def carefully_buy_cat(buyer: CatBuyer) -> Cat:
    try:
        cat = buyer.buy_cat()
    except BankruptError as e:
        print(e, "I guess next month, you will eat pasta...")
        buyer.beg_for_money(10_000)
        cat = buyer.buy_cat()
    return cat

buyer = CatBuyer(money=1)
cat = carefully_buy_cat(buyer)
cat.be_cute()

Sorry, you're broke... I guess next month, you will eat pasta...
Your bank despise you.
Passive cuteness always ON. Nothing to do in particular


**Good extension:** The new class only throws subclasses of the exceptions thrown by the base class.

In [71]:
# Good extension
class FancyBankruptError(BankruptError):
    ...


class FancyBuyer(CatBuyer):
    def __init__(self, money=None):
        super().__init__(money=9001)
        if money is not None:
            self.money = money

    def buy_cat(self) -> BengalCat:
        cat = BengalCat()
        if self.money < cat.price:
            raise FancyBankruptError(
                "Sorry, you may be fancy but you're not THAT rich..."
            )
        self.money -= cat.price
        print("I'm ruined, but at least I have a fancy cat ┬──┬◡ﾉ(° -°ﾉ)")
        return cat


In [72]:
# Client code
buyer = FancyBuyer(5000)
cat = carefully_buy_cat(buyer)
cat.be_cute()

Sorry, you may be fancy but you're not THAT rich... I guess next month, you will eat pasta...
Your bank despise you.
I'm ruined, but at least I have a fancy cat ┬──┬◡ﾉ(° -°ﾉ)
Passive cuteness always ON. Nothing to do in particular


**Bad extension:** The new subclass throws a totally different type of exception than the base class.

In [77]:
class NotACatError(Exception):
    ...


class BlindBuyer(CatBuyer):
    def __init__(self, money=None):
        super().__init__(money=500)
        if money is not None:
            self.money = money

    def buy_cat(self) -> Cat:
        cat = Animal(300, "???")
        if self.money < cat.price:
            raise BankruptError("I can't count my money since I cannot see anything!!!")
        if cat.sound != "purr":
            raise NotACatError("Wait it has WAY TOO MANY TEETH... ABORT!!!")
        print("I don't know what I'm doing. But I have a cat (?!!)")
        return cat


In [78]:
# Client code
buyer = BlindBuyer(5000)
cat = carefully_buy_cat(buyer)
cat.be_cute()

NotACatError: Wait it has WAY TOO MANY TEETH... ABORT!!!

### Final Example: Documents

#### Before fix

The base hierarchy looks like the following:

![Before refactoring](docs/diagrams/out/__WorkspaceFolder__/various_modules/docs/diagrams/src/read_only_documents/OriginalReadOnlyDocument.png)

In [3]:
class ReadOnlyError(Exception):
    ...

class Document:
    def __init__(self, data: str, filename: str):
        self.data = data
        self.filename = filename

    def open(self) -> None:
        print("open the following document:", self.filename)
    
    def save(self) -> None:
        print("save", self.filename)

class ReadOnlyDocument(Document):
    def save(self) -> None:
        raise ReadOnlyError("Unable to save read-only file.")

class Project:
    def __init__(self, documents: list[Document]):
        self.documents = documents

    def open_all(self) -> None:
        for doc in self.documents:
            doc.open()

    def save_all(self) -> None:
        for doc in self.documents:
            if not isinstance(doc, ReadOnlyDocument):
                doc.save()

In [4]:
writable_doc = Document("a", "a")
readonly_doc = ReadOnlyDocument("b", "b")

project = Project([writable_doc, readonly_doc])
project.open_all()
project.save_all()

open the following document: a
open the following document: b
save a


#### After fix

The new organization is like this:

![After fixing](docs/diagrams/out/__WorkspaceFolder__/various_modules/docs/diagrams/src/read_only_documents/AfterRefactorReadOnlyDocument.png)

In [2]:
class Document:
    def __init__(self, data: str, filename: str):
        self.data = data
        self.filename = filename

    def open(self) -> None:
        print("open document:", self.filename)

class WritableDocument(Document):
    def save(self) -> None:
        print("save document:", self.filename)

class Project:
    def __init__(self, read_only_docs: list[Document], writable_docs: list[Document]):
        self.read_only_docs = read_only_docs
        self.writable_docs = writable_docs

    def open_all(self) -> None:
        for doc in self.read_only_docs + self.writable_docs:
            doc.open()

    def save_all(self) -> None:
        for doc in self.writable_docs:
            doc.save()

In [3]:
writable_doc = WritableDocument("a", "a")
readonly_doc = Document("b", "b")

project = Project(read_only_docs=[readonly_doc], writable_docs=[writable_doc])
project.open_all()
project.save_all()

open document: b
open document: a
save document: a
